**Read Precribed Trajectories**

In [ ]:
from skope_extraction_utils import * 

In [ ]:
traj_file = "/neurospin/metric/ZainebAmor/February2024/dim3_i_RadialIO_P0.75_N192x192x128_FOV0.192x0.192x0.128_Nc8_Ns2688_c25_d2__D9M9Y2021T1017_reproject.bin"
FOV, M, Ns, OS, n_shots  = get_traj_params(traj_file)
kspace_loc_norm = get_samples(traj_file, 0.01/OS, Ns*OS,kmax=(M[0]/(2*FOV[0]), M[1]/(2*FOV[1]), M[2]/(2*FOV[2])))
kspace_loc_unorm = get_kspace_loc_from_gradfile(traj_file, 0.01/OS, Ns*OS)[0]

In [ ]:
kspace_loc_unorm.shape, FOV, M, Ns, OS, n_shots

**Read Measured Data**

In [ ]:
import mat73
import scipy.io
meas_file = "/neurospin/metric/ZainebAmor/February2024/cr170417_13022024/R5.mat"
ndummy=900
meas_traj, meas_k0 = read_meas_matrix(meas_file, ndummy)
meas_traj.shape

**Synch**

In [ ]:
s_range=1000
e_range=3000
nshots=n_shots
corr, ind, delay, corr_ind, delay_ind = find_skope_delay(s_range,e_range, meas_traj, kspace_loc_unorm, Ns*OS, nshots)
plt.plot(delay, corr)
plt.xlabel('K-space point shift')
plt.ylabel('Correlation between the measured \n and prescribed k-space  positions')
plt.title('Correlation between the measured and prescribed \nk-space  positions according  to the shift')
print(delay_ind, corr_ind)

In [ ]:
nsamp=Ns*OS
dim=3
Kmax= (M[0]/(2*FOV[0]), M[1]/(2*FOV[1]), M[2]/(2*FOV[2]))
new_meas_traj, meas_k0 = extract_synch_trajs_and_k0(meas_traj, meas_k0,dim,nsamp, delay_ind, Kmax)

In [ ]:
alpha = 23
rot_matrix =  np.array([[np.cos(alpha*np.pi/180), 0, np.sin(alpha*np.pi/180)],[0, 1, 0],[-np.sin(alpha*np.pi/180), 0, np.cos(alpha*np.pi/180)]])
new_meas_traj = np.dot(new_meas_traj, rot_matrix)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
j=5
plt.plot(kspace_loc_norm[nsamp*j:nsamp*(j+1),0], label='th')
plt.plot(new_meas_traj[nsamp*j:nsamp*(j+1),0], label='meas')
plt.subplot(1,3,2)
plt.plot(kspace_loc_norm[nsamp*j:nsamp*(j+1),1], label='th')
plt.plot(new_meas_traj[nsamp*j:nsamp*(j+1),1], label='meas')
plt.subplot(1,3,3)
plt.plot(kspace_loc_norm[nsamp*j:nsamp*(j+1),2], label='th')
plt.plot(new_meas_traj[nsamp*j:nsamp*(j+1),2], label='meas')
plt.legend()

In [ ]:
path_traj = "/neurospin/metric/ZainebAmor/February2024/cr170417_13022024/kmatrices/kxyz_R5"
path_k0="/neurospin/metric/ZainebAmor/February2024/cr170417_13022024/kmatrices/k0_R5"
save_trajs_and_k0(new_meas_traj, meas_k0, path_traj, path_k0)

In [ ]:
ecc_file = "/neurospin/metric/ZainebAmor/February2024/cr170417_13022024/ecc_raw/ECCPhase.mat"

In [ ]:
n_rep = 125
ecc = ECCPhase_prep(ecc_file, nshots=n_shots*n_rep, nsamp=nsamp)
print(ecc.shape, new_meas_traj.shape, meas_k0.shape)
np.save("/neurospin/metric/ZainebAmor/February2024/cr170417_13022024/ecc_raw/ecc", ecc)